<a href="https://colab.research.google.com/github/Soniartha/Trading-Strategy/blob/main/LSTMs_Jason_Brownlee__Univariate_Multisteps_with_Encoder_Decoder_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
  #Import library
  from numpy import array
  from keras.models import Sequential
  from keras.layers import Dense
  from keras.layers import LSTM
  from keras.layers import RepeatVector
  from keras.layers import TimeDistributed

In [3]:
#Define split function to convert sequence data into X and y structure
#Define Split Function to Convert sequence data into X and y structure
def split_sequence(sequence, n_steps, n_steps_out):
  X, y = list (), list ()
  for i in range(len(sequence)):
    end_ix = i + n_steps
    out_end_ix = end_ix + n_steps_out
    if out_end_ix > len(sequence):
      break
    seq_x, seq_y = sequence[i : end_ix], sequence[end_ix:out_end_ix]
    X.append(seq_x)
    y.append(seq_y)
  return array(X), array(y)

In [4]:
#Define sequence
series = [1,2,3,4,5,6,7,8,9,10]

In [5]:
#Define n_steps
n_steps, n_steps_out = 3,2

In [6]:
#Split sequence into X and y structure
X, y = split_sequence(series, n_steps, n_steps_out)

In [7]:
#Explore X and y dataset
print(X.shape, y.shape)
for i in range(len(X)):
  print(X[i], y[i])

(6, 3) (6, 2)
[1 2 3] [4 5]
[2 3 4] [5 6]
[3 4 5] [6 7]
[4 5 6] [7 8]
[5 6 7] [8 9]
[6 7 8] [ 9 10]


In [8]:
#Reshape from [samples, timesteps] into [samples, timesteps, feature]
n_feature = 1
X = X.reshape((X.shape[0], X.shape[1], n_feature))
y = y.reshape((y.shape[0], y.shape[1], n_feature))

In [11]:
#Define model
model = Sequential()
model.add(LSTM(100, activation= 'relu', input_shape = (n_steps, n_feature)))
model.add(RepeatVector(n_steps_out))
model.add(LSTM(100, activation= 'relu', return_sequences= True))
model.add(TimeDistributed(Dense(1)))
model.compile(optimizer = 'adam', loss = 'mse')

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100)               40800     
_________________________________________________________________
repeat_vector (RepeatVector) (None, 2, 100)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 2, 100)            80400     
_________________________________________________________________
time_distributed (TimeDistri (None, 2, 1)              101       
Total params: 121,301
Trainable params: 121,301
Non-trainable params: 0
_________________________________________________________________


In [ ]:
#Model fit
model.fit(X, y, epochs= 1000, verbose= 2)

In [25]:
print(X.shape, y.shape)
X[5, :, :]

(6, 3, 1) (6, 2, 1)


array([[6],
       [7],
       [8]])

In [32]:
#Demostrate prediction
x_input = array([8, 9, 10])
x_input = x_input.reshape((1, n_steps, n_feature))
y_hat = model.predict(x_input, verbose= 0)
print(y_hat)

[[[11.037136]
  [12.019609]]]
